# 完整的模型验证(测试、 demo)流程

In [2]:
#导入第三方库
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import torchvision

In [3]:
imgs_path = "test_imgs/dog.png"
image = Image.open(imgs_path)
print(image)

<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=355x237 at 0x1A074B44610>


In [4]:
#png格式的图片是4通道，除了RGB通道外，还有透明度通道，所以需要调用image = image.convert('RGB')保留图片的三个颜色通道
image = image.convert('RGB')

In [5]:
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((32,32)),
                                            torchvision.transforms.ToTensor()])
image = transform(image)
print(image.shape)

torch.Size([3, 32, 32])


In [27]:
#网络结构
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x

#加载已经训练的模型
# model = torch.load("models_train/model_1.pth")
neural_model = Model()
model.load_state_dict(torch.load("models_train/model_1.pth", weights_only= True))
print(neural_model)

Model(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [30]:
image = torch.reshape(image, [1, 3, 32, 32])
#设置模型为验证状态
model.eval()
#不让梯度参与计算
with torch.no_grad():
    output = model(image)
print(output)
print(output.argmax(1))

tensor([[-1.9275, -0.0207,  0.5383,  0.8544,  0.9382,  0.8855,  1.1639,  0.5515,
         -1.4541, -0.5712]])
tensor([6])
